<a href="https://colab.research.google.com/github/keripikkaneboo/Hands-On-Machine-Learning-O-Reilly-/blob/main/05.%20Chapter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bab 5: Support Vector Machines (SVM)

Bab ini memperkenalkan **Support Vector Machines (SVM)**, sebuah model yang sangat kuat dan serbaguna yang dapat melakukan klasifikasi linier maupun non-linier, regresi, dan bahkan deteksi *outlier*.

* **Ide Fundamental SVM**: Konsep utama di balik SVM adalah **klasifikasi margin besar** (*large margin classification*). SVM mencoba menemukan "jalan" (*street*) terluas yang memisahkan dua kelas. Tujuannya adalah untuk memaksimalkan *margin*, yaitu jarak antara batas keputusan dan instance terdekat dari setiap kelas. Instance yang berada di tepi "jalan" ini disebut **support vectors**, karena merekalah yang "mendukung" atau menentukan batas keputusan.

* **Hard Margin vs. Soft Margin**:
    * ***Hard Margin Classification***: Mengharuskan semua instance berada di luar "jalan" dan di sisi yang benar. Pendekatan ini hanya berfungsi jika data dapat dipisahkan secara linier dan sangat sensitif terhadap *outlier*.
    * ***Soft Margin Classification***: Pendekatan yang lebih fleksibel dengan mencari keseimbangan antara menjaga "jalan" selebar mungkin dan membatasi pelanggaran margin (*margin violations*), yaitu instance yang berakhir di tengah jalan atau bahkan di sisi yang salah. Tingkat fleksibilitas ini dikendalikan oleh *hyperparameter* `C`.

* **Klasifikasi SVM Non-Linier**:
    * Untuk data yang tidak dapat dipisahkan secara linier, SVM dapat menggunakan trik matematika yang disebut **kernel trick**. Trik ini memungkinkan SVM untuk mendapatkan hasil yang sama seolah-olah kita menambahkan banyak fitur non-linier (seperti fitur polinomial tingkat tinggi) tanpa benar-benar harus melakukannya, sehingga menghindari ledakan jumlah fitur secara komputasi.
    * **Kernel Populer**:
        * ***Polynomial Kernel***: Untuk data dengan pola polinomial.
        * ***Gaussian RBF Kernel***: Kernel yang sangat kuat dan berfungsi dengan baik pada berbagai jenis dataset.

* **Regresi SVM**:
    * SVM juga dapat digunakan untuk tugas regresi. Tujuannya dibalik: alih-alih mencoba membuat "jalan" terluas yang memisahkan dua kelas, SVM Regresi mencoba memasukkan sebanyak mungkin instance *ke dalam* "jalan" sambil membatasi pelanggaran margin (instance di luar jalan).
    * Lebar "jalan" ini dikendalikan oleh *hyperparameter* `epsilon` (ε).

### 1. Klasifikasi SVM Linier
Kita akan menggunakan dataset Iris untuk mendemonstrasikan SVM. Kita akan membuat pengklasifikasi biner untuk mendeteksi apakah bunga adalah *Iris-Virginica* atau bukan.

```python
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

# Memuat dataset Iris
iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = (iris["target"] == 2).astype(np.float64)  # 1.0 jika Iris-Virginica, else 0.0

# Membuat pipeline dengan scaler dan model LinearSVC
# Hyperparameter C mengontrol trade-off margin
# Nilai C yang lebih kecil menghasilkan margin yang lebih lebar tetapi lebih banyak pelanggaran
svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(C=1, loss="hinge", random_state=42)),
    ])

svm_clf.fit(X, y)

# Membuat prediksi
print("Prediksi untuk petal length=5.5, petal width=1.7:", svm_clf.predict([[5.5, 1.7]]))
```

### 2. Klasifikasi SVM Non-Linier dengan Kernel
Untuk data non-linier, kita bisa menggunakan `SVC` dengan kernel. Kita akan menggunakan dataset `moons` sebagai contoh.

```python
from sklearn.datasets import make_moons
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC
import matplotlib.pyplot as plt

# Membuat dataset moons
X, y = make_moons(n_samples=100, noise=0.15, random_state=42)

def plot_dataset(X, y, axes):
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "bs")
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "g^")
    plt.axis(axes)
    plt.grid(True, which='both')
    plt.xlabel(r"$x_1$")
    plt.ylabel(r"$x_2$")

# --- Model dengan Kernel Polinomial ---
poly_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
    ])
poly_kernel_svm_clf.fit(X, y)


# --- Model dengan Kernel Gaussian RBF ---
rbf_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
    ])
rbf_kernel_svm_clf.fit(X, y)

# Catatan: Kode untuk mem-plot decision boundary tidak ditampilkan di sini agar ringkas,
# namun tersedia di notebook Jupyter dari buku.
print("Model dengan kernel polinomial dan RBF telah dilatih.")
```

### 3. Regresi SVM
SVM juga bisa digunakan untuk regresi. `SVR` adalah kelas yang setara dengan `SVC` untuk regresi.

```python
from sklearn.svm import SVR

# Membuat data acak non-linier
np.random.seed(42)
m = 100
X = 2 * np.random.rand(m, 1) - 1
y = (0.2 + 0.1 * X + 0.5 * X**2 + np.random.randn(m, 1) / 10).ravel()

# Melatih model SVR dengan kernel polinomial
# epsilon (ε) mengontrol lebar "jalan"
svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X, y)

# Melatih model SVR lain dengan kernel RBF
svm_rbf_reg = SVR(kernel="rbf", C=0.1, gamma=0.1)
svm_rbf_reg.fit(X, y)


def plot_svm_regression(svm_reg, X, y, axes):
    x1s = np.linspace(axes[0], axes[1], 100).reshape(100, 1)
    y_pred = svm_reg.predict(x1s)
    plt.plot(x1s, y_pred, "k-", linewidth=2, label=r"$\hat{y}$")
    plt.plot(x1s, y_pred + svm_reg.epsilon, "k--")
    plt.plot(x1s, y_pred - svm_reg.epsilon, "k--")
    plt.plot(X, y, "bo")
    plt.axis(axes)


plt.figure(figsize=(9, 4))
plt.subplot(121)
plot_svm_regression(svm_poly_reg, X, y, [-1, 1, 0, 1])
plt.title("SVR dengan Kernel Polinomial (degree=2)")
plt.ylabel(r"$y$")

plt.subplot(122)
plot_svm_regression(svm_rbf_reg, X, y, [-1, 1, 0, 1])
plt.title("SVR dengan Kernel RBF (C=0.1, gamma=0.1)")
plt.show()
```
Plot di atas menunjukkan bagaimana SVR mencoba memasukkan sebanyak mungkin instance ke dalam "jalan" (di antara garis putus-putus) yang lebarnya dikontrol oleh `epsilon`.

